In [ ]:
import os
import h5py
import time

import numpy as np
import matplotlib.pyplot as plt

from extra_data import open_run, RunDirectory, H5File
from extra_data.read_machinery import find_proposal

In [ ]:
from dynflatfield import DynamicFlatFieldCorrectionCython as DynamicFlatFieldCorrection
from dynflatfield import FileDynamicFlatFieldProcessor

In [ ]:
propno = 2919
runno_dark = 59
runno_flat = 40

camno = 2
n_components = 20

downsample_factors = (2, 4)

fn = f"pca_cam{camno}_d{runno_dark}_f{runno_flat}_r{n_components}.h5"
karabo_id = f"SPB_EHD_HPVX2_{camno}"
cam_source = f"{karabo_id}/CAM/CAMERA:daqOutput"
image_key = "data.image.pixels"

dffc = DynamicFlatFieldCorrection.from_file(fn, karabo_id, downsample_factors)

In [ ]:
runno = 30

propdir = find_proposal(f"p{propno:06d}")
rundir = os.path.join(propdir, f"raw/r{runno:04d}")

run = RunDirectory(rundir)

In [ ]:
tm0 = time.monotonic()

proc = FileDynamicFlatFieldProcessor(dffc, 32, cam_source, image_key)

proc.start_workers()
proc.run(run)
proc.join_workers()

tm1 = time.monotonic()
print(f"Time: {tm1 - tm0:.2f} s")

In [ ]:
def process_run(dffc, run, source, image_key="data.image.pixels"):
    results = []
    weights = []
    warnflag = []

    cam_data = run.select([(source, image_key)])
    for tid, data in cam_data.trains():
        images = data[source].get(image_key)
        if images is None:
            continue
        
        images_corr, w, warnflag_i = dffc.process(images, False)
        results.append(images_corr)
        weights.append(w)
        warnflag.append(warnflag_i)
        
    return results, weights, warnflag

tm0 = time.monotonic()

#images = run[cam_source, 'data.image.pixels'].ndarray()
r, w, warnflag = process_run(dffc, run, cam_source)

tm1 = time.monotonic()
print(f"Time: {tm1-tm0:.2f} s")

In [ ]:
t = 35
i = 92
fig, (ax1, ax2) =plt.subplots(1,2, figsize=(16, 6))

ax1.matshow(proc.rdr.results[t][i], vmin=0, vmax=5)
ax2.matshow(r[t][i], vmin=0, vmax=5)

plt.show()

im = plt.matshow(proc.rdr.results[t][i]-r[t][i])
plt.colorbar(im)
plt.show()
np.sum(np.abs(proc.rdr.results[t][i]-r[t][i]))

In [ ]:
w1 = np.concatenate(w).reshape(-1, 128, 20)
np.linalg.norm(w1[t,i]-w1[t-1,i])
#plt.plot(w1[t-1,i])

In [ ]:
r1 = np.concatenate(proc.rdr.results)
r2 = np.concatenate(r)

In [ ]:
x = np.sum(np.abs(r1 - r2), axis=(1,2))

In [ ]:
ic = plt.matshow(x.reshape(-1,128))#, vmax=1000)
plt.colorbar(ic)



In [ ]:
plt.hist(x.ravel(), 100)
plt.semilogy()
plt.show()